In [13]:
import pandas as pd

data_pd = pd.read_csv('./data/Churn_Modelling.csv')
data_pd = data_pd.drop(labels=['RowNumber','CustomerId','Surname'], axis=1)
data_pd.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder_gender = LabelEncoder()
data_pd['Gender'] = label_encoder_gender.fit_transform(data_pd['Gender'])

data_pd.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data_pd[['Geography']])

onehot_encoder_geo.get_feature_names_out(['Geography'])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [16]:
geo_pd = pd.DataFrame(geo_encoder.toarray(), columns= onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_pd.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [17]:
data_pd = pd.concat([data_pd.drop('Geography', axis=1), geo_pd], axis=1)
data_pd.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
import pickle

with open('./data/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('./data/onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [19]:
X = data_pd.drop('Exited', axis = 1)
Y = data_pd['Exited']
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:

with open('./data/scalar.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [30]:
import tensorflow as tf
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
# Build a Sequential model with 2 hidden layers
model = Sequential(
    [
        Dense(64, activation = 'relu', input_shape = X_train[1].shape),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

# Print summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 

In [25]:

model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [36]:
import datetime

log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
earlystop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights= True)


In [37]:
model.fit(
    X_train, 
    Y_train, 
    validation_data=(X_test, Y_test), 
    callbacks=[tensorboard_callback, earlystop_callback], 
    epochs=100
    )

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2325 - accuracy: 0.9053 - val_loss: 0.4132 - val_accuracy: 0.8460
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2325 - accuracy: 0.9053 - val_loss: 0.4132 - val_accuracy: 0.8460
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2315 - accuracy: 0.9046 - val_loss: 0.4177 - val_accuracy: 0.8425
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2315 - accuracy: 0.9046 - val_loss: 0.4177 - val_accuracy: 0.8425
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2306 - accuracy: 0.9043 - val_loss: 0.4181 - val_accuracy: 0.8430
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2306 - accuracy: 0.9043 - val_loss: 0.4181 - val_accuracy: 0.8430
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2317 - accuracy: 0.9040 - val_loss: 0.4335 - val_accuracy: 0.8200

In [38]:
model.save('./data/model.h5')

c:\Users\sajee\Documents\Abilash\AI Practice\deep-learning\ann-basic\annenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [52]:
%tensorboard --logdir logs/fit20251020-002531

Reusing TensorBoard on port 6010 (pid 10972), started 0:00:02 ago. (Use '!kill 10972' to kill it.)